In [19]:
from __future__ import print_function

import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import mlflow
import mlflow.sklearn

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")
print("Tracking URI: ", mlflow.tracking.get_tracking_uri())

In [ ]:
experiment_name = "sklearn_logistic_regression"
print("experiment_name: ", experiment_name)
mlflow.set_experiment(experiment_name)

client = mlflow.tracking.MlflowClient()
experiment_id = client.get_experiment_by_name(experiment_name).experiment_id
print("experiment_id: ", experiment_id)

In [2]:
np.random.seed(137)

In [16]:
X, y = datasets.make_classification(n_samples=1000, 
                                    n_features=2,
                                    n_informative=2,
                                    n_redundant=0,
                                    n_repeated=0,
                                    n_classes=2)

In [40]:
def run(X, y, penalty='l2'):
    
    solver = "saga"
    if penalty is "elasticnet":
        l1_ratio = 0.5
    else:
        l1_ratio = None
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    
    with mlflow.start_run(run_name="jupyter") as run:
        lr = LogisticRegression(penalty=penalty, solver=solver, l1_ratio=l1_ratio)
        lr.fit(X_train, y_train)
        score_train = lr.score(X_train, y_train)
        score_test = lr.score(X_test, y_test)

        print("train score: ", score_train)
        print("test score: ", score_test)
        print("hyperparameters: ", lr.get_params())
    
        print("runId: ", run.info.run_id)
        print("artifact_uri: ", mlflow.get_artifact_uri())
        mlflow.log_metrics({"training score": score_train, "test score": score_test})
        mlflow.log_params(lr.get_params())
        
#         if log_model:
#             mlflow.sklearn.log_model(lr, "model")

In [41]:
run(X, y, "l2")

train score:  0.8493333333333334
test score:  0.888
hyperparameters:  {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'saga', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [42]:
run(X, y, "l1")

train score:  0.8506666666666667
test score:  0.888
hyperparameters:  {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l1', 'random_state': None, 'solver': 'saga', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [43]:
run(X, y, "elasticnet")

train score:  0.8506666666666667
test score:  0.888
hyperparameters:  {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': 0.5, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'elasticnet', 'random_state': None, 'solver': 'saga', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [ ]:
sk_model = mlflow.sklearn.load_model("runs:/e10a8ad6c3074573b0a5c6a83d657af2/model")

In [ ]:
sk_model

In [ ]:
sk_model.predict(np.array([-2, -1, 0, 1, 2, 1]).reshape(-1, 1))